In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
from scipy.stats import norm
from IPython.display import display

pd.set_option('display.float_format', '{:,.2f}'.format)

In [2]:
data = pd.read_excel('Repaso.xlsx')
data = data.set_index(data.Date)
data = data.drop(columns=['Date'])
data

,QQQ,VOO,VXX,ITOT
Date,,,,
2023-01-03,263.33,345.87,55.96,83.44
2023-01-04,264.58,348.36,54.44,84.14
2023-01-05,260.44,344.56,54.88,83.14
2023-01-06,267.63,352.39,53.60,85.04
2023-01-09,269.36,352.14,53.88,85.05
...,...,...,...,...
2023-12-04,386.32,419.74,17.28,100.78
2023-12-05,387.29,419.61,17.16,100.58
2023-12-06,385.05,417.86,17.24,100.24


# 1)

In [3]:
def VaR_CVaR_Stocks(data, n_stocks, conf, long):
     
     valor_posicion = n_stocks * data.iloc[-1]
     weights = [valor_posicion.iloc[i]/valor_posicion.sum() for i in range(len(n_stocks))]  
     rt_assets = data.pct_change().dropna()
     rt_port = rt_assets.mul(weights).sum(axis=1)

     var = np.percentile(rt_port,100-conf) if long else np.percentile(rt_port,conf)
     cvar = rt_port[rt_port < var].mean() if long else rt_port[rt_port > var].mean()
          
     var_m = var * valor_posicion.sum()
     cvar_m = cvar * valor_posicion.sum()
     
     resultados_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var * 100, cvar * 100], "Monetario ($)": [var_m, cvar_m]})

     return resultados_df

In [4]:
positions = [3967, 1576, 1023, 2032] # En mismo orden que Stocks

long = False #True o False

conf = 99 #[0, 100]

VaR_CVaR_Stocks(data, positions, conf, long)['Porcentual (%)'][1] * (-1)

-2.5893295200536026

In [5]:
VaR_CVaR_Stocks(data, positions, conf, long)['Monetario ($)'][1] * (-1)

-63322.615799510204

In [6]:
rt = data.pct_change().dropna()
rt.corr()

,QQQ,VOO,VXX,ITOT
QQQ,1.00,0.92,-0.62,0.90
VOO,0.92,1.00,-0.73,0.99
VXX,-0.62,-0.73,1.00,-0.74
ITOT,0.90,0.99,-0.74,1.00


# 2)

# 3)

In [7]:
data = pd.read_excel('Repaso.xlsx', sheet_name=2)
data = data.set_index(data.Date)
data = data.drop(columns=['Date'])
data

,CHFMXN=X,JPYMXN=X,USDMXN=X
Date,,,
2023-01-02,21.02,0.15,19.45
2023-01-03,21.05,0.15,19.46
2023-01-04,20.73,0.15,19.41
2023-01-05,20.85,0.15,19.39
2023-01-06,20.64,0.14,19.32
...,...,...,...
2023-12-05,20.02,0.12,17.45
2023-12-06,19.84,0.12,17.36
2023-12-07,19.76,0.12,17.29


In [8]:
def VaR_CVaR_Fx(data, positions, conf, long):
     port = data * positions 
     total = port.sum(axis=1)
     rt = total.pct_change().dropna()

     var = np.percentile(rt,100-conf) if long else np.percentile(rt,conf)
     cvar = rt[rt < var].mean() if long else rt[rt > var].mean()

     var_m = var * total.iloc[-1]
     cvar_m = cvar * total.iloc[-1]

     resultados_df = pd.DataFrame({"Métrica": ["VaR", "CVaR"], "Porcentual (%)": [var, cvar], "Monetario ($)": [var_m, cvar_m]})

     return resultados_df

In [9]:
positions = [2300000, 23500000, 2500000] # En mismo orden que Fx

long = True #True o False

NC = 95 #[0, 100]

VaR_CVaR_Fx(data, positions, NC, long)['Monetario ($)'][1] / 1000000

-1.1926150248956378

# 4)

In [10]:
# Escribir listas en formato ['S', 'K', 'r', 'sigma', 'T', 'N']

info_call = [[462, 465, 0.0528, 0.17, 1/12, 10],
             [462, 470, 0.0528, 0.16, 1/12, 12],
             [462, 475, 0.0528, 0.15, 1/12, 17],
             [462, 465, 0.0523, 0.21, 3/12, 18],
             [462, 470, 0.0523, 0.20, 3/12, 20],
             [462, 475, 0.0523, 0.19, 3/12, 12],
             [462, 465, 0.0505, 0.22, 6/12, 16],
             [462, 470, 0.0505, 0.21, 6/12, 14],
             [462, 475, 0.0505, 0.20, 6/12, 11]]

info_put = [[462, 450, 0.0528, 0.24, 1/12, 26],
            [462, 445, 0.0528, 0.22, 1/12, 18],
            [462, 440, 0.0528, 0.20, 1/12, 14],
            [462, 450, 0.0523, 0.25, 3/12, 21],
            [462, 445, 0.0523, 0.24, 3/12, 26],
            [462, 440, 0.0523, 0.23, 3/12, 32],
            [462, 450, 0.0505, 0.26, 6/12, 17],
            [462, 445, 0.0505, 0.25, 6/12, 35],
            [462, 440, 0.0505, 0.24, 6/12, 43]]

In [11]:
call_blsdelta = lambda S, k, r, sigma, T: norm.cdf((np.log(S/k) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T)))

put_blsdelta = lambda S, k, r, sigma, T: np.abs(norm.cdf((np.log(S/k) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))) - 1)

def delta_hedge(info_call, info_put):

    df_call = pd.DataFrame(info_call, columns=['S', 'K', 'r', 'sigma', 'T', 'N'])
    df_put = pd.DataFrame(info_put, columns=['S', 'K', 'r', 'sigma', 'T', 'N'])
    
    deltas_call_list = [call_blsdelta(*df_call.values[i][0:-1]) for i in range(len(df_call))]
    deltas_put_list = [put_blsdelta(*df_put.values[i][0:-1]) for i in range(len(df_call))]

    return np.dot(df_call.N, deltas_call_list) - np.dot(df_put.N, deltas_put_list)

In [12]:
net_1 = delta_hedge(info_call, info_put)
print(f'Vender en corto {net_1:,.6f} miles de títulos')

Vender en corto -6.971954 miles de títulos
